unet is the network that will given some noisified image and we also indicate the netork what is the amount of noise added to the this image. the model has to predict how much noise is there and how to remove it. this unet is a bunch of convolution that will reduce the size of the image with each step by increasing the number of features.and then we did the reverse step just like we did with the decoder in VAE. again we will do some convolution with the residual blocks with attention block etc. one big difference is that we need to tell our unet not only the the image with noise , not only the ammount with noise , the timestep at which the noise was added but also the prompt because we also need to tell the unet what is our prompt because we want to tell him how we want our output image because thereare many ways to denoise the initial noise. So if we want our initial noise to become a dog we want to tell him we wannt a dog. so, unet has to know what is the prompt and he has to relate this prompt with rest of the imformation.And the best way to combine two different stuff e.g.image with text is cross attention. cross attention allow us to calculate the attention between two sequences in which query is the first sequence and the key and values are comming from another sequence. lets see how it works.

the first thing we will do is create a new  class called diffusion because this will be our diffusion model. we will build from top down. So, we first define the difffusion class and then we build each block one by one.

[1] we need to give the unet not only the noisified image but also the time step at which it was noisified. so the unet has need some way to understand this time step so this time step which is a number is converted into an embedding by this particular model in out our code called TimeEmbedding.

[2] so why we are here more features(320) than the starting, in diagram we can see the last layerof unet actually we need to go back to the same number of features . when we built the decoder, it didnt build the final number of features that we need which is 4, so we need an additional output layer to go back to the original size of the features. this is the job of UNET_OutputLayer

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
from attention import SelfAttention, CrossAttention



#lets build first the time embedding, we build it after diffusion class, something that encodes information about time step in which we are, it is made of two linear layers
class TimeEmbedding(nn.Module):

  def __init__(self, n_embd: int):
    super().__init__()
    self.linear_1 = nn.Linear(n_embd, 4*n_embd)
    self.linear_2 = nn.Linear(4*n_embd, n_embd) #this is the reason why it becomes 1280, which is 4 times 320

  def forward(self, x: torch.Tensor) -> torch.Tensor:
    #x:(1,320)[the input size is 1,320]
    #what we do we first apply this first layer linear_1, then we apply silu fnction
    x = self.linear_1(x)
    x = F.silu(x)
    x = self.linear_2(x)
    #the output is (1,1280)
    return x

#now we need to  build residual block we build it after unet output layer block
class UNET_ResidualBlock(nn.Module):

  def __init__(self, in_channels: int, out_channels: int, n_time=1280): #this is the embedding of the time step as we remember in the time embedding we transform into an embedding size of one to 1280
    super().__init__()
    #we have groupnorm, conv and linear in this block pritty similar to vae residual block
    self.groupnorm_feature = nn.GroupNorm(32, in_channels)
    self.conv_feature = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
    self.linear_time = nn.Linear(n_time, out_channels) #we have linear for time embedding


    #then we have another norm, thn conv
    self.groupnorm_merged = nn.GroupNorm(32, out_channels)
    self.conv_merged = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)


    #residual connection , we did it before
    if in_channels == out_channels:
      self.residual_layer = nn.Identity()
    else:
      self.residual_layer = nn.Conv2d(in_channels, out_channels, kernel_size=1, padding=0)

    #now we have to define the forward, the idea of (residual block#) is tht in diagram we have 3 input, time embedding, latent , we need find a way to combine the three information together
    #so the unet needs to learn to detect the noise present in the noisified image at a particular time step using a particular prompt as a condition
    #which means the model needs to recognise this time embedding and needs to relate this time embedding with the latents , so we are doing this residual block
    #we are relating latent with time embedding so tht the output will depend on the combination of both (noise and time step), this will also be done with the context of cross attention tht we build next
  def forward(self, feature, time):
    #feature:(batch_size,in_channel, hight, width)
    #time:(1,1280)

    residue=feature

    feature=self.groupnorm_feature(feature)
    feature=F.silu(feature)
    feature=self.conv_feature(feature)
    time=F.silu(time)
    time=self.linear_time(time)

    #here we are merging the latents with the time embedding but the time embedding doesnt have the batch size n channel dim so we add it here with unsqueeze
    #and we merge them then norm the merged connection, this is why it called merged. then we apply activation function, conv
    merged=feature+time.unsqueeze(-1).unsqueeze(-1)
    merged=self.groupnorm_merged(merged)
    merged=F.silu(merged)
    merged=self.conv_merged(merged)
    #finally we apply residual connection
    return merged+ self.residual_layer(residue)

# after residual block
class UNET_AttentionBlock(nn.Module):

  def __init__(self, n_head: int, n_embd: int, d_context=768):
    super().__init__()
    channels= n_head*n_embd #this layer will make sense when we define the forward method

    self.groupnorm =nn.GroupNorm(32, channels, eps=1e-6)
    self.conv_input=nn.Conv2d(channels, channels, kernel_size=1, padding=0)

    self.layernorm_1=nn.LayerNorm(channels)
    self.attention=SelfAttention(n_head, n_embd, in_proj_bias=False)

    self.layernorm_2=nn.LayerNorm(channels)
    self.attention_2=CrossAttention(n_head, channels, d_context, in_proj_bias=False)

    self.layernorm_3=nn.LayerNorm(channels)
    self.linear_geglu_1=nn.Linear(channels, 4*channels*2)
    self.linear_geglu_2=nn.Linear(4*channels, channels)

    self.conv_output=nn.Conv2d(channels, channels, kernel_size=1, padding=0)

    #now we can build the forward method
  def forward(self, x, context):
    #x is our latent, we have (batch size, features, height, width)
    #then we have our context which is our prompt , it has batch size,seq_len, dim

    #first thing we will do is normalization just like transformer, we will take the input so our latents and we apply the norm and the conv
    #actually in transformer there is no conv , only norm

    #this is called long reside because it will applied at the end
    residue_long=x
    x=self.groupnorm(x) #we apply norm tht doesnt change the size of the tensor
    x=self.conv_input(x) #x =self.conv input of x, which also doesnt change the size of tensor

    n,c,h,w=x.shape# thewetake shape which is the batch size, the number of features , height and width

    #(batch size, features, height, width)->(batch size, features, height*width)
    x=x.view(n,c,h*w)

    #we transpose bcz we want to apply cross attention, first self then cross

    x=x.transpose(-1,-2)
    #now we do norm +self attention with skip connection
    residue_short=x
    x=self.layernorm_1(x)
    x=self.attention_1(x)
    x+=residue_short#first residu connection

    residue_short=x #because now we apply cross attention
    x=self.layernorm_2(x)
    x=self.attention_2(x,context)
    x+=residue_short

    residue_short=x

    #now we do norm +cross attention with skip connection, we will do cross attention later
    x=self.layernorm_2(x)

    #then we apply cross attn
    self.attention_2(x,context) #cross between laten n prompt

    x+=residue_short

    residue_short=x

    #finally just like attention transformer , we have feed fwd layer with geglu activation
    x=self.layernorm_3(x)

    x,gate = self.linear_geglu_1(x).chunk(2, dim=-1)
    x= x*F.gelu(gate) #wedo elementwise multiplication , basically this process is same as transformer in stable diffsion model
    #these are special activation function tht involve lot of parameters, but we use use this , not the other , they just saw it works better in this type of application
    self.llinear_geglu_2(x)
    x+= residue_short #skip connection

    x=x.transpose(-1,-2) #reverse the previous transpose
    x=x.view(n,c,h,w) #reverse the multiplication

    #finally we will apply the long skip connection tht we defined here at the begining , only if the size match, if size doesnt match we apply only self.conv_output
    return self.conv_output(x)+residue_long



#lets build all those block we didnt build, first bild upsample, we build it after unet decoder
class UpSample(nn.Module):

  def __init__(self, channels: int):
    super().__init__()
    self.conv = nn.Conv2d(channels, channels, kernel_size=3, padding=1)

  def forward(self, x):
    #(batch_size, Features, Height,width)-> (batch_size, Features, Height*2,width*2), because we are going to use upsampling
    x = F.interpolate(x, scale_factor=2, mode="nearest")#it will double the size basically, 2:38:18
    return self.conv(x)


#its better to build switch sequential saparately, otherwise it would be too many mess in unet block so we decided to bild switch sequential block
class SwitchSequential(nn.Sequential):

  def forward(self,x: torch.Tensor, context: torch.Tensor, time: torch.Tensor) -> torch.Tensor:
    #we will apply one by one
    for layer in self:
      if isinstance(layer, UNET_AttentionBlock):
        x = layer(x, context) #this attention block basically compute the cross attention between our latentsand the prompts , thats why
      elif isinstance(layer, UNET_ResidualBlock):
        x = layer(x, time)
      else:
        x = layer(x)
    return x



#the next thing we need to build is unet, we bild it after time embedding block, the unet will require many blocks, lets just build the unet itself and then we build each block of unet
class UNET(nn.Module):

  def __init__(self):
    super().__init__()
    #as we can see unet is made of one encoder branch, bottle neck and decoder part and then skip connections between encoder and decoder, so the output of each layerof which step of the encoder is connected to the same stepof the decoder on the other side.
    # we start from left side which is encoder which is a list of modules
    self.encoder=nn.module([
        #(batch_size, 4, Height/8, width/8)
        SwitchSequential(nn.Conv2D(4,320, kernel_size=3, padding=1)),#basically this sequence is in which given a list of layers. we will apply tem one by one, we can think of it as a sequential but it can recognise what are the parameters of each of them
        SwitchSequential(UNET_ResidualBlock(320,320), UNET_AttentionBlock(8,40)),
        SwitchSequential(UNET_ResidualBlock(320,320), UNET_AttentionBlock(8,40)),
        SwitchSequential(nn.Conv2D(320,320, kernel_size=3,stride=2 padding=1)),
        SwitchSequential(UNET_ResidualBlock(320,640), UNET_AttentionBlock(8,80)),#this attention block takes number of head, this 8 will indicate number of head, 80 indicates the embedding size
        SwitchSequential(UNET_ResidualBlock(640,640), UNET_AttentionBlock(8,80)),
        #(batch_size, 640, Height/16, width/16) -> #(batch_size, 640, Height/32, width/32)
        SwitchSequential(nn.Conv2D(640,640, kernel_size=3,stride=2 padding=1)),
        SwitchSequential(UNET_ResidualBlock(640,1280), UNET_AttentionBlock(8,160)),
        SwitchSequential(UNET_ResidualBlock(1280,1280), UNET_AttentionBlock(8,80)),
        SwitchSequential(nn.Conv2D(1280,1280, kernel_size=3,stride=2 padding=1)),
        SwitchSequential(UNET_ResidualBlock(1280,1280)),
        SwitchSequential(UNET_ResidualBlock(1280,1280)), # we keep reducing the size of the image but we keep increasing the features of each pixels
    ])

#then we build thebottle neck, this is a sequence of residual block, cross attention and another residual block
    self.bottle_neck=SwitchSequential(
        UNET_ResidualBlock(1280,1280),

        UNET_AttentionBlock(8,160)

        UNET_ResidualBlock(1280,1280)
    )
    #then we build the decoder, in decoder we will do the opposite what we do in encoder, reduce the number of featres but increase theimage size
    self.decoder=nn.moduleList([
        #(batch_size,2560,Height/64, width/64)->(batch_size,1280,Height/64, width/64)
        SwitchSequential(UNET_ResidualBlock(2560,1280)), #input of the decoder is the output of the bottleneck, skip connection is the reason
        SwitchSequential(UNET_ResidualBlock(2560,1280)),
        SwitchSequential(UNET_ResidualBlock(2560,1280), UpSample(1280)),#to increase the size of the image we do up sample , ame concept as vae decoder but different
        SwitchSequential(UNET_ResidualBlock(2560,1280), UNET_AttentionBlock(8,160)),
        SwitchSequential(UNET_ResidualBlock(2560,1280), UNET_AttentionBlock(8,160)),
        SwitchSequential(UNET_ResidualBlock(1920,1280), UNET_AttentionBlock(8,160), UpSample(1280)),
        SwitchSequential(UNET_ResidualBlock(1920,640), UNET_AttentionBlock(8,80)),
        SwitchSequential(UNET_ResidualBlock(1280,640), UNET_AttentionBlock(8,80)),
        SwitchSequential(UNET_ResidualBlock(960,640), UNET_AttentionBlock(8,80), UpSample(1280)),
        SwitchSequential(UNET_ResidualBlock(960,320), UNET_AttentionBlock(8,40)),
        SwitchSequential(UNET_ResidualBlock(640,320), UNET_AttentionBlock(8,80)),
        SwitchSequential(UNET_ResidualBlock(640,320),UNET_AttentionBlock(8,40)),
    ])

#now we have to define unet output layer , we build it after upsampling block, it has a group norm of 32 size of the group 32, conv
class UNET_OutputLayer(nn.Module):

  def __init__(self,in_channels:int, out_channels:int):
    super().__init__()
    self.groupnorm= nn.GroupNorm(32, in_channels)
    self.conv= nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
#final layer needs to convert this shape#(batch_size,320,Height/8, width/8) into this shape (batch_size,4,Height/8, width/8)
  def forward(self, x):
    #so we have a input x: batch_size,320,Height/8, width/8), first we apply grouporm
    x=self.groupnorm(x)
    x=F.silu(x)
    x=self.conv(x)
    #(batch_size,4,Height/8, width/8)
    return x


#first we define the diffusion class. class diffsion is basicaly our unet
class Diffusion(nn.Module):
    def __init__(self):
        self.time_embedding=TimeEmbedding(320) # this is made of time embedding,something that we will define later, 320 is the size of time embedding more info in text section[1]
        self. unet= UNET() #then we build the unet
        self.final= UNET_OutputLayer() #finally the output layer of unet, we will later see what is this letter used

    def forward(self,latent:torch.Tensor, context torch.Tensor, time:torch.Tensor): # as we remember unet will receive the latent z, the output of vae, it will also receive the context (out prompt) which is also a tensor, finally it will receive the time at which it was noisified
        #lets define the sizes, latent here is a batch size 4 because 4 is the output of the encoder, height and width divided by 8
        #then the context which we already converted with the clip encoder which will be a bitsize by seq len by dimension where dimension is 768 that de defined before.
        #time , we will define it later how it build, but each embedding its a number with a embedding of vector of size 320
        #latent(Batch_size ,4, height/8, width/8 )
        #context(Batch_size, Seq_len, Dim)
        #time(1,320)


        #the first thing we do we convet this time into an embedding, and we will see later this time is actually just like the positional encoding ofthe transformer model
        #its actually a number that is multiplied by sin and cosine. it works for the transformer so we can also use the same positionalencoder to convey the information of the time, so its  tell the model at which step we arrived in the denoisification.

        #(1,320)->(1,1280) this one will convert the tensor of 320 into a tensor of 1280
        time=self.time_embedding(time)

        #the unet will convert out latent into another latent, so we will not change the size
        #(batch_size, 4, height/8, width/8)->(Batch_size, 320, height/8, width/8)
        #^this is the output of the variational autoencoder, which first becomes batch 320 features through this unet, check the txet part for more info[2]
        output=self.unet(latent, context,time)
        #this one will go from (Batch_size, 320, height/8, width/8) to original size of of the unet (Batch_size) [see the video 2:17:57 for more explanation]
        output= self.final(output)
        #(Batch_size, 320, height/8, width/8)
        return output
